In [ ]:

import tensorflow as tf
from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation, add, GlobalAveragePooling2D,SeparableConv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Reshape
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.layers import Reshape, GlobalAveragePooling2D, Dense

# Load datasets
(mnist_train_X, mnist_train_y), (mnist_test_X, mnist_test_y) = mnist.load_data()
(fmnist_train_X, fmnist_train_y), (fmnist_test_X, fmnist_test_y) = fashion_mnist.load_data()
(cifar10_train_X, cifar10_train_y), (cifar10_test_X, cifar10_test_y) = cifar10.load_data()

# Preprocess data
def preprocess_data(X, y, num_classes):
    X = X.astype('float32') / 255.0
    y = to_categorical(y, num_classes)
    return X, y

# MNIST and FMNIST: expand dimensions to match CIFAR-10
mnist_train_X = np.expand_dims(mnist_train_X, axis=-1)
mnist_test_X = np.expand_dims(mnist_test_X, axis=-1)
fmnist_train_X = np.expand_dims(fmnist_train_X, axis=-1)
fmnist_test_X = np.expand_dims(fmnist_test_X, axis=-1)

mnist_train_X, mnist_train_y = preprocess_data(mnist_train_X, mnist_train_y, 10)
mnist_test_X, mnist_test_y = preprocess_data(mnist_test_X, mnist_test_y, 10)
fmnist_train_X, fmnist_train_y = preprocess_data(fmnist_train_X, fmnist_train_y, 10)
fmnist_test_X, fmnist_test_y = preprocess_data(fmnist_test_X, fmnist_test_y, 10)
cifar10_train_X, cifar10_train_y = preprocess_data(cifar10_train_X, cifar10_train_y, 10)
cifar10_test_X, cifar10_test_y = preprocess_data(cifar10_test_X, cifar10_test_y, 10)

# Define model architectures

def lenet5(input_shape, num_classes):
    input = Input(shape=input_shape)
    x = Conv2D(6, kernel_size=(5, 5), activation='relu')(input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(16, kernel_size=(5, 5), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(120, activation='relu')(x)
    x = Dense(84, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input, outputs=output)
    return model

def alexnet(input_shape, num_classes):
    input = Input(shape=input_shape)
    
    # Initial Conv Layer
    x = Conv2D(96, kernel_size=(11, 11), strides=(1, 1), padding='same', activation='relu')(input)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    # Second Conv Layer
    x = Conv2D(256, kernel_size=(5, 5), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    # Third Conv Layer
    x = Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    # Flatten and Dense Layers
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output)
    return model


from tensorflow.keras.layers import Concatenate

def googlenet(input_shape, num_classes):
    input = Input(shape=input_shape)
    x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same', activation='relu')(input)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = Conv2D(192, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    def inception_module(x, filters):
        branch1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)
        branch2 = Conv2D(filters[1], (1, 1), padding='same', activation='relu')(x)
        branch2 = Conv2D(filters[2], (3, 3), padding='same', activation='relu')(branch2)
        branch3 = Conv2D(filters[3], (1, 1), padding='same', activation='relu')(x)
        branch3 = Conv2D(filters[4], (5, 5), padding='same', activation='relu')(branch3)
        branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
        branch4 = Conv2D(filters[5], (1, 1), padding='same', activation='relu')(branch4)
        x = Concatenate(axis=-1)([branch1, branch2, branch3, branch4])
        return x

    x = inception_module(x, [64, 96, 128, 16, 32, 32])
    x = inception_module(x, [128, 128, 192, 32, 96, 64])
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception_module(x, [192, 96, 208, 16, 48, 64])
    x = inception_module(x, [160, 112, 224, 24, 64, 64])
    x = inception_module(x, [128, 128, 256, 24, 64, 64])
    x = inception_module(x, [112, 144, 288, 32, 64, 64])
    x = inception_module(x, [256, 160, 320, 32, 128, 128])
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception_module(x, [256, 160, 320, 32, 128, 128])
    x = inception_module(x, [384, 192, 384, 48, 128, 128])
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input, outputs=output)
    return model
       

def vggnet(input_shape, num_classes):
    input = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(input)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input, outputs=output)
    return model

def resnet(input_shape, num_classes):
    def residual_block(x, filters, strides=(1, 1)):
        shortcut = x
        x = Conv2D(filters, (3, 3), strides=strides, padding='same', activation='relu')(x)
        x = BatchNormalization()(x)
        x = Conv2D(filters, (3, 3), padding='same', activation=None)(x)
        x = BatchNormalization()(x)
        if strides != (1, 1) or x.shape[-1] != filters:
            shortcut = Conv2D(filters, (1, 1), strides=strides, padding='same', activation=None)(shortcut)
            shortcut = BatchNormalization()(shortcut)
        x = add([x, shortcut])
        x = Activation('relu')(x)
        return x

    input = Input(shape=input_shape)
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = residual_block(x, 128, strides=(2, 2))
    x = residual_block(x, 128)
    x = residual_block(x, 256, strides=(2, 2))
    x = residual_block(x, 256)
    x = residual_block(x, 512, strides=(2, 2))
    x = residual_block(x, 512)
    x = GlobalAveragePooling2D()(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input, outputs=output)
    return model

def xception(input_shape, num_classes):
    input = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), strides=(2, 2), padding='same', activation='relu')(input)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    residual = Conv2D(128, (1, 1), strides=(2, 2), padding='same', activation=None)(x)
    residual = BatchNormalization()(residual)
    x = SeparableConv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = SeparableConv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = add([x, residual])
    residual = Conv2D(256, (1, 1), strides=(2, 2), padding='same', activation=None)(x)
    residual = BatchNormalization()(residual)
    x = SeparableConv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = SeparableConv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = add([x, residual])
    residual = Conv2D(728, (1, 1), strides=(2, 2), padding='same', activation=None)(x)
    residual = BatchNormalization()(residual)
    x = SeparableConv2D(728, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = SeparableConv2D(728, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = add([x, residual])
    for _ in range(8):
        residual = x
        x = SeparableConv2D(728, (3, 3), padding='same', activation='relu')(x)
        x = BatchNormalization()(x)
        x = SeparableConv2D(728, (3, 3), padding='same', activation='relu')(x)
        x = BatchNormalization()(x)
        x = SeparableConv2D(728, (3, 3), padding='same', activation='relu')(x)
        x = BatchNormalization()(x)
        x = add([x, residual])
    residual = Conv2D(1024, (1, 1), strides=(2, 2), padding='same', activation=None)(x)
    residual = BatchNormalization()(residual)
    x = SeparableConv2D(1024, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = SeparableConv2D(1024, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = add([x, residual])
    x = SeparableConv2D(1536, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = SeparableConv2D(2048, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input, outputs=output)
    return model

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Dense, Reshape, Multiply
from tensorflow.keras.models import Model

def senet(input_shape, num_classes):
    def squeeze_excite_block(x, ratio=16):
        init = x
        filters = x.shape[-1]
        se_shape = (1, 1, filters)

        # Global average pooling
        se = GlobalAveragePooling2D()(x)
        se = Reshape(se_shape)(se)

        # Fully connected layers
        se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
        se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

        # Scaling
        se = Reshape((1, 1, filters))(se)  # Ensure the shape matches x
        x = Multiply()([init, se])
        return x

    input = Input(shape=input_shape)
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    for filters in [64, 128, 256, 512]:
        for _ in range(3):
            x = Conv2D(filters, (3, 3), padding='same', activation='relu')(x)
            x = BatchNormalization()(x)
            x = squeeze_excite_block(x)
    x = GlobalAveragePooling2D()(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input, outputs=output)
    return model

# Function to plot and print metrics
def evaluate_model(model_name, model, train_X, train_y, test_X, test_y):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_X, train_y, epochs=10, batch_size=128, validation_data=(test_X, test_y), verbose=1)
    
    predictions = model.predict(test_X)
    y_true = np.argmax(test_y, axis=1)
    y_pred = np.argmax(predictions, axis=1)
    
    print(f"Metrics for {model_name} on {dataset_name} dataset")
    print(classification_report(y_true, y_pred))
    print("Confusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.arange(num_classes), yticklabels=np.arange(num_classes))
    plt.title(f'Confusion Matrix for {model_name}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()
    
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

input_shape = (32, 32, 3)  # CIFAR-10 images are 32x32x3
num_classes = 10
dataset_name = 'CIFAR-10'

# Define the models
models = {
#     'LeNet-5': lenet5,
#     'AlexNet': alexnet,
#     'GoogLeNet': googlenet,
#     'VGGNet': vggnet,
#     'ResNet': resnet,
#     'Xception': xception,
#     'SENet': senet
}

# Evaluate each model
for model_name, model_function in models.items():
    print(f"Evaluating {model_name} on {dataset_name} dataset...")
    model = model_function(input_shape, num_classes)
    accuracy = evaluate_model(model_name, model, cifar10_train_X, cifar10_train_y, cifar10_test_X, cifar10_test_y)
    print(f"Accuracy for {model_name} on {dataset_name} dataset: {accuracy:.4f}\n")


Evaluating AlexNet on CIFAR-10 dataset...
Epoch 1/10
 82/391 ━━━━━━━━━━━━━━━━━━━━ 4:02 786ms/step - accuracy: 0.1152 - loss: 2.5020